In [2]:
import duckdb
import pandas as pd

# Load the cleaned economist metadata
econ = pd.read_csv("economists_master_dataset.csv")

# Extract article title from URL
econ["page_title"] = econ["article_url_x"].str.split("/wiki/").str[-1]

econ.head()


,name_x,article_url_x,qid,summary_x,name_y,article_url_y,summary_y,gender_raw,birthdate_raw,citizenship_raw,occupation_raw,field_raw,birth_year,gender_qid,page_title
0,Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,Q272731,"Edith Abbott (September 26, 1876 – July 28, 19...",Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,"Edith Abbott (September 26, 1876 – July 28, 19...","[{'entity-type': 'item', 'numeric-id': 6581072...","[{'time': '+1876-09-26T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1876.0,Q6581072,Edith_Abbott
1,Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,Q718581,"Kamer Daron Acemoğlu (born September 3, 1967) ...",Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,"Kamer Daron Acemoğlu (born September 3, 1967) ...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1967-09-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 43, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1967.0,Q6581097,Daron_Acemoglu
2,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Q7001311,Nicola Acocella (born 3 July 1939) is an Itali...,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Nicola Acocella (born 3 July 1939) is an Itali...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1939-07-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 172579,...","[{'entity-type': 'item', 'numeric-id': 188094,...",NaN,1939.0,Q6581097,Nicola_Acocella
3,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Q8073604,Zoltan J. Acs (born 1947) is an American econo...,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Zoltan J. Acs (born 1947) is an American econo...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1947-03-09T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1947.0,Q6581097,Zoltan_Acs
4,Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,Q518021,"Henry Carter Adams (December 31, 1851 – August...",Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,"Henry Carter Adams (December 31, 1851 – August...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1851-12-31T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1851.0,Q6581097,Henry_Carter_Adams


In [3]:
econ = pd.read_csv("economists_master_dataset.csv")

In [6]:
conn = duckdb.connect()
conn

In [7]:
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

In [8]:
database_url = "https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb"

In [9]:
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb


In [10]:
query = "PRAGMA show_tables"
result = conn.sql(query)
result

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘

In [11]:
table_name = "data_table"
query = f"PRAGMA table_info('web_db.{table_name}');"

# We can apply the method .df() to the result of the query to convert it into a dataframe
column_info_df = conn.sql(query).df()
column_info_df

,cid,name,type,notnull,dflt_value,pk
0,0,date,DATE,False,None,False
1,1,country,VARCHAR,False,None,False
2,2,country_code,VARCHAR,False,None,False
3,3,project,VARCHAR,False,None,False
4,4,page_id,BIGINT,False,None,False
5,5,article,VARCHAR,False,None,False
6,6,qid,VARCHAR,False,None,False
7,7,pageviews,BIGINT,False,None,False


In [12]:
query_1 = """
SELECT * FROM data_table
LIMIT 10;
"""
result_1 = conn.sql(query_1).df() # after executing, convert to df for better printout

result_1

,date,country,country_code,project,page_id,article,qid,pageviews
0,2024-01-01,United States,US,en.wikipedia,100580,"Selma,_Alabama",Q79941,117
1,2024-01-01,United States,US,en.wikipedia,10058256,Jimmy_John_Liautaud,Q24041548,143
2,2024-01-01,United States,US,en.wikipedia,1008381,Cheerwine,Q5089372,300
3,2024-01-01,United States,US,en.wikipedia,10095918,New_York_Fashion_Week,Q1113829,117
4,2024-01-01,United States,US,en.wikipedia,10122474,Stephanie_Edwards,Q7608269,195
5,2024-01-01,United States,US,en.wikipedia,101615,Tocumen_International_Airport,Q1420178,117
6,2024-01-01,United States,US,en.wikipedia,1016778,Ãrebro,Q25732,123
7,2024-01-01,United States,US,en.wikipedia,1023183,Kidd-class_destroyer,Q246916,124
8,2024-01-01,United States,US,en.wikipedia,1027629,Long_Duk_Dong,Q55621207,305
9,2024-01-01,United States,US,en.wikipedia,10338,Excommunication,Q49822,237


In [18]:
econ_qids = duckdb.from_df(econ[["qid"]])

In [20]:
conn.register("econ_df", econ)

In [21]:
conn.execute("""
    CREATE TEMP TABLE econ_q AS 
    SELECT qid FROM econ_df
""")

In [30]:
query = """
SELECT 
    e.qid,
    SUM(d.pageviews) AS total_views_2024,
    d.date
FROM econ_q e
LEFT JOIN data_table d
    ON e.qid = d.qid
GROUP BY e.qid
ORDER BY total_views_2024 DESC;
"""
views_2024 = conn.execute(query).df()
views_2024.head()


BinderException: Binder Error: column "date" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(date)" if the exact value of "date" is not important.

LINE 5:     d.date
            ^

In [23]:
econ_with_views = econ.merge(views_2024, on="qid", how="left")
econ_with_views.head()

,name_x,article_url_x,qid,summary_x,name_y,article_url_y,summary_y,gender_raw,birthdate_raw,citizenship_raw,occupation_raw,field_raw,birth_year,gender_qid,total_views_2024
0,Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,Q272731,"Edith Abbott (September 26, 1876 – July 28, 19...",Edith Abbott,https://en.wikipedia.org/wiki/Edith_Abbott,"Edith Abbott (September 26, 1876 – July 28, 19...","[{'entity-type': 'item', 'numeric-id': 6581072...","[{'time': '+1876-09-26T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1876.0,Q6581072,NaN
1,Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,Q718581,"Kamer Daron Acemoğlu (born September 3, 1967) ...",Daron Acemoglu,https://en.wikipedia.org/wiki/Daron_Acemoglu,"Kamer Daron Acemoğlu (born September 3, 1967) ...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1967-09-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 43, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1967.0,Q6581097,83090.0
2,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Q7001311,Nicola Acocella (born 3 July 1939) is an Itali...,Nicola Acocella,https://en.wikipedia.org/wiki/Nicola_Acocella,Nicola Acocella (born 3 July 1939) is an Itali...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1939-07-03T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 172579,...","[{'entity-type': 'item', 'numeric-id': 188094,...",NaN,1939.0,Q6581097,NaN
3,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Q8073604,Zoltan J. Acs (born 1947) is an American econo...,Zoltan Acs,https://en.wikipedia.org/wiki/Zoltan_Acs,Zoltan J. Acs (born 1947) is an American econo...,"[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1947-03-09T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1947.0,Q6581097,NaN
4,Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,Q518021,"Henry Carter Adams (December 31, 1851 – August...",Henry Carter Adams,https://en.wikipedia.org/wiki/Henry_Carter_Adams,"Henry Carter Adams (December 31, 1851 – August...","[{'entity-type': 'item', 'numeric-id': 6581097...","[{'time': '+1851-12-31T00:00:00Z', 'timezone':...","[{'entity-type': 'item', 'numeric-id': 30, 'id...","[{'entity-type': 'item', 'numeric-id': 188094,...","[{'entity-type': 'item', 'numeric-id': 8134, '...",1851.0,Q6581097,NaN


In [24]:
econ_with_views.to_csv('merged_econ_data.csv', index=False)